<a href="https://colab.research.google.com/github/aurotripathy/RL-Experiments/blob/main/qwen_vl_expt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Ensure you change the runtime to A100 - 80GB

In [ ]:
%pip install transformers

In [ ]:
!nvidia-smi

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Qwen3VLMoeForConditionalGeneration, AutoProcessor
import torch

# default: Load the model on the available device(s)
model = Qwen3VLMoeForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-30B-A3B-Instruct", dtype="auto", device_map="cuda"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen3VLMoeForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-30B-A3B-Instruct",
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-30B-A3B-Instruct")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]


# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

In [ ]:
from IPython.display import Image, display
import requests

# Assuming the image is the first item in the first message's content
image_url = messages[0]['content'][0]['image']

# Download the image from the URL
response = requests.get(image_url)
with open("temp_image.jpeg", "wb") as f:
    f.write(response.content)

# Display the image
display(Image(filename="temp_image.jpeg"))

In [ ]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)